In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_trade, cancel_trade, place_accept, close_trading
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))

#close_trading(client, 693, storeAppID, creator)

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator, token_id=asset_id)
# storeApp.create_app()
# storeAppID = storeApp.app_id

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID
print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Store App ID: 727
Store App Address: 5YFHLDU7ELKNT3N25YKEXEHALZL6C3UEZLKZOSMICARY3ZBXDEPRKAKYWU


In [7]:

appID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction QLVZMH72I5E7NXH6SYAZAOKHXSHCZHQSWKK7VGE2Y5FIORIFSPGQ confirmed in round 44414.
App ID: 1191
App Address: IKFUUZDNRXCZSVEWGFKEGSNHXBYXFFBMYRQP6X3EOTEAD4L3NHWCSSFKNQ


Alice is setting up and funding token auction...

In [8]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_id=asset_id,
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]
print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction MC42FZ4OJKJATGB7GTVQDBN5VJTHGT7PRGIWNHATCJQXMA6EDLHQ confirmed in round 44416.
Alice's algo balance:  20020878032475  algos


In [9]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 201000, 499: 0}


In [10]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10011054680720  algos
Carla is placing bid for:  1000000  algos


In [11]:
place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice)

print("Carla bid", asset_id)

bidder Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM opt in app 1191
Waiting for confirmation...
Transaction NGE4T43M2NX5TANP7HOWVKSRI5LS3ZGNMEY764VZNZVIQHNIB72A confirmed in round 44419.
store_app_id 727
token_txn: {'sender': 'Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM', 'fee': 1000, 'first_valid_round': 44417, 'last_valid_round': 45417, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 'group': None, 'lease': None, 'type': 'axfer', 'rekey_to': None, 'receiver': 'IKFUUZDNRXCZSVEWGFKEGSNHXBYXFFBMYRQP6X3EOTEAD4L3NHWCSSFKNQ', 'amount': 10, 'index': 499, 'close_assets_to': None, 'revocation_target': None}


AlgodHTTPError: TransactionPool.Remember: transaction YSZALJXWRXQHIOJDNCGFL5FMD3OPAVKT4FT3D6UNZXZIIMFT6BTQ: logic eval error: assert failed pc=287

In [ ]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_trade(client, appID, seller)
print("Carla cancel bid", asset_id)

In [ ]:
place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice)

place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=15, price=bidPrice)



In [ ]:
place_accept(client, creator, appID, bidder, seller.get_address())

Alice is closing out the auction....

In [ ]:
sleep(5)

close_trading(client, appID, creator, [asset_id])

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
storeAppState = get_app_global_state(client, 727)
print(storeAppState)
print(encoding.encode_address(storeAppState[b"C"]))